## 코로나와 온라인쇼핑 네이버 뉴스 토픽모델링

- 참고코드 : https://bumcrush.tistory.com/155

- 네이버뉴스_코로나온라인쇼핑_뉴스크롤링 파일위치 (C:/Users/jminw/Untitled Folder/coron)
- 2020.09.01 ~ 2020.09.10 뉴스

In [1]:
import pandas as pd
import numpy as np

df= pd.DataFrame(pd.read_csv('coron.csv', encoding='cp949'))
df.head()

,Unnamed: 0,years,company,title,contents,link
0,1,2020.09.09.,OhmyNews,"코로나19 시대, 친환경 '온라인 녹색매장' 열린다",제공대형마트 등에서 운영하던 '녹색매장'이 온라인 인터파크(www.interpark...,https://news.naver.com/main/read.nhn?mode=LSD&...
1,2,2020.09.09.,헤럴드경제,"“친환경 제품, 온라인 녹색매장서 구매하세요”",환경산업기술원 녹색제품 기획전환경부 산하 한국환경산업기술원(원장 유제철·사진)은 9...,https://news.naver.com/main/read.nhn?mode=LSD&...
2,3,2020.09.10.,ZDNet Korea,"쿠팡, 소상공인 온라인 판로 지원","농림축산식품부, 한국농수산식품유통공사와 농공상기업 돕는 기획전 진행. 쿠팡이 코로나...",https://news.naver.com/main/read.nhn?mode=LSD&...
3,4,2020.09.09.,financial news,"서울시, 소상공인·전통시장 온라인 진출 지원한다",서울시는 온라인쇼핑 운영 경험이 많지 않은 소상공인과 전통시장 상인을 대상으로 온라...,https://news.naver.com/main/read.nhn?mode=LSD&...
4,5,2020.09.09.,뉴스1,"서울시, '언택트 시대' 소상공인·전통시장 상인 온라인 진출 지원",소상공인 300곳 쿠팡 등 입점 지원…판매수수료 2~5%p 인하전통시장 온라인시장 ...,https://news.naver.com/main/read.nhn?mode=LSD&...


In [2]:
import re 
# '한글'을 제외한 다른 문자 모두 제거
remove_except_ko = re.compile(r"[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]")
def preprocess(text):
    text = re.sub(remove_except_ko,' ',text).strip()
    return text

df['title'] = df['title'].map(lambda x : preprocess(x))
df['contents'] = df['contents'].map(lambda x : preprocess(x))
df.head()

,Unnamed: 0,years,company,title,contents,link
0,1,2020.09.09.,OhmyNews,코로나 시대 친환경 온라인 녹색매장 열린다,제공대형마트 등에서 운영하던 녹색매장 이 온라인 인터파크 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
1,2,2020.09.09.,헤럴드경제,친환경 제품 온라인 녹색매장서 구매하세요,환경산업기술원 녹색제품 기획전환경부 산하 한국환경산업기술원 원장 유제철 사진 은 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
2,3,2020.09.10.,ZDNet Korea,쿠팡 소상공인 온라인 판로 지원,농림축산식품부 한국농수산식품유통공사와 농공상기업 돕는 기획전 진행 쿠팡이 코로나...,https://news.naver.com/main/read.nhn?mode=LSD&...
3,4,2020.09.09.,financial news,서울시 소상공인 전통시장 온라인 진출 지원한다,서울시는 온라인쇼핑 운영 경험이 많지 않은 소상공인과 전통시장 상인을 대상으로 온라...,https://news.naver.com/main/read.nhn?mode=LSD&...
4,5,2020.09.09.,뉴스1,서울시 언택트 시대 소상공인 전통시장 상인 온라인 진출 지원,소상공인 곳 쿠팡 등 입점 지원 판매수수료 인하전통시장 온라인시장 ...,https://news.naver.com/main/read.nhn?mode=LSD&...


In [3]:
# Mecab
from konlpy.tag import Mecab
# 선언
mecab= Mecab()

In [4]:
stop_words = ['은','는','이','가','을','를','께서','에서','이다','에게','으로','이랑','까지','부터','하다',
             '기자','뉴스','무단','전재','및','재배포','금지','저작권','사진','김병기','앵커','이연춘','연합뉴스',
             '파이낸셜뉴스','습니다','입니다']
stopPos = ['JKS','JKC','JKG','JKO','JKB','JKV','JKQ','JC','JX',
           'SF','SE','SS','SP','SSO','SSC','SC','SY','SH','SL','SN',
           'MM','MAG','MAJ']
def token_mecab(text):
    text = mecab.pos(text)
    text = [i for i in text if len(i[0])>1]
    text = [i for i in text if i[0] not in stop_words]
    text = [i[0] for i in text if i[1] not in stopPos]
    return text

In [5]:
token_mecab('쿠팡이 코로나19로 판로를 잃고 어려움을 겪고 있는 소상공인과 지역농가를 위해 발 벗고 나섰다.쿠팡은 중소상공인희망재단, 서울시와 손잡고')

['쿠팡',
 '코로나',
 '판로',
 '어려움',
 '소상',
 '공인',
 '지역',
 '농가',
 '위해',
 '나섰',
 '쿠팡',
 '중소',
 '상공',
 '희망',
 '재단',
 '서울시',
 '손잡']

In [6]:
def make_tokens(df):
    df['tokens'] = ''
    tokens_list=[]
    for i in range(len(df)):
        token = token_mecab(df['contents'][i])
        df['tokens'][i] = ' '.join(token)
    return df

df = make_tokens(df)

C:\Data\envs\py37\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
df.head()

,Unnamed: 0,years,company,title,contents,link,tokens
0,1,2020.09.09.,OhmyNews,코로나 시대 친환경 온라인 녹색매장 열린다,제공대형마트 등에서 운영하던 녹색매장 이 온라인 인터파크 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,공대 마트 운영 녹색 온라인 인터 파크 열렸 올해 코로나 인해 온라인 쇼핑몰 이용자...
1,2,2020.09.09.,헤럴드경제,친환경 제품 온라인 녹색매장서 구매하세요,환경산업기술원 녹색제품 기획전환경부 산하 한국환경산업기술원 원장 유제철 사진 은 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,환경 산업 기술원 녹색 제품 기획 전환 경부 산하 한국 환경 산업 기술원 원장 유제...
2,3,2020.09.10.,ZDNet Korea,쿠팡 소상공인 온라인 판로 지원,농림축산식품부 한국농수산식품유통공사와 농공상기업 돕는 기획전 진행 쿠팡이 코로나...,https://news.naver.com/main/read.nhn?mode=LSD&...,농림 축산 식품 한국 농수산 식품 유통 공사 농공 기업 기획 진행 쿠팡 코로나 판로...
3,4,2020.09.09.,financial news,서울시 소상공인 전통시장 온라인 진출 지원한다,서울시는 온라인쇼핑 운영 경험이 많지 않은 소상공인과 전통시장 상인을 대상으로 온라...,https://news.naver.com/main/read.nhn?mode=LSD&...,서울시 온라인 쇼핑 운영 경험 소상 공인 전통 시장 상인 대상 온라인 시장 진출 지...
4,5,2020.09.09.,뉴스1,서울시 언택트 시대 소상공인 전통시장 상인 온라인 진출 지원,소상공인 곳 쿠팡 등 입점 지원 판매수수료 인하전통시장 온라인시장 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,상공 쿠팡 입점 지원 판매 수수료 인하 전통 시장 온라인 시장 진입 는다 온라인 장...


In [9]:
# 공백으로 토큰 분리
def my_tokenizer(text):
    return text.split()

In [10]:
df['tokens']

0     공대 마트 운영 녹색 온라인 인터 파크 열렸 올해 코로나 인해 온라인 쇼핑몰 이용자...
1     환경 산업 기술원 녹색 제품 기획 전환 경부 산하 한국 환경 산업 기술원 원장 유제...
2     농림 축산 식품 한국 농수산 식품 유통 공사 농공 기업 기획 진행 쿠팡 코로나 판로...
3     서울시 온라인 쇼핑 운영 경험 소상 공인 전통 시장 상인 대상 온라인 시장 진출 지...
4     상공 쿠팡 입점 지원 판매 수수료 인하 전통 시장 온라인 시장 진입 는다 온라인 장...
5     모바일 홈쇼핑 포털 애플리케이션 쇼핑 모아 운영 버즈 공동 대표 남상협 김성국 쇼핑...
6     광복 자갈치시장 송관 만들 전통 시장 온라인 판매 롯데 백화점 광복 부산 자갈치 시...
7     부산 지역 최대 수산물 시장 자갈치 시장 롯데 백화점 광복 코로나 불황 이겨 고자 ...
8     라이브 공연 커머스 결합 라이브 라떼 출시 공연 면서 아이돌 판매 가능 서울시 지역...
9     미래학자 제이슨 초대 세계 변화 농업 미래 전략 대담 농림 축산 식품 대한민국 농업...
10    대구시 사회 경제 기업 판로 지원 위해 온라인 쇼핑 마켓 개설 구시 동구 사회 경제...
11    프레시 지역 농가 지원 온라인 유통 업계 최근 신종 코로나 바이러스 감염증 코로나 ...
12    특산물 판매 문화 행사 영동군 주최 프로젝트 충북 영동군 신종 코로나 바이러스 감염...
13    삼성 직원 겨냥 이벤트 다음 중순 운영 협력 업체 물품 대금 조기 지급 삼성전자 직...
14    농축 수산물 음식 서비스 지난 오후 서울 노원구 롯데마트 중계 직원 온라인 주문 상...
15    오전 서울 중구 북창동 순두부 관계자 배달 코로나 확산 음식 조리 거나 배달 경우 ...
16    국내 코로나 사태 지속 가운데 악천후 겹치 면서 온라인 생필품 구매 사람 늘어났 악...
17    면역력 관심 반영 수출 내수 급증 투입 집중 육성 북도 최근 면역 강화식품 

In [11]:
type(df['tokens'])

pandas.core.series.Series

In [12]:
s = df['tokens']

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

tfidf_vect = TfidfVectorizer(tokenizer = my_tokenizer)
tfidf = tfidf_vect.fit_transform(s)

#모델 선언
lda = LatentDirichletAllocation(n_components=5)
lda_output = lda.fit_transform(tfidf)

In [14]:
import pyLDAvis
import pyLDAvis.sklearn

pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda, tfidf, tfidf_vect, mds='tsne')
pyLDAvis.display(vis)